# Verifying zeros of the Fisher system

We begin by defining the function $f$ and its derivative $Df$.

In [1]:
using LinearAlgebra # Load the LinearAlgebra package
using Polynomials   # Load the Polynomials package

In [2]:
function f(x, lambda)
  N = length(x) - 1

  x2 = zeros(size(x))  # The quadratic term
  f_x = zeros(size(x)) # The computed f(x)

  for k = 0:N
    # Compute the quadratic term
    for k1 = -N:N
      k2 = k - k1
      if abs(k2) <= N
        x2[k + 1] += x[abs(k1) + 1] * x[abs(k2) + 1]
      end
    end

    f_x[k + 1] = (lambda - k^2) * x[k + 1] - lambda * x2[k + 1]
  end

  return f_x
end

f (generic function with 1 method)

In [3]:
function Df(x, lambda)
  N = length(x) - 1

  Df_x = zeros(N + 1, N + 1) # Jacobian matrix Df(x)

  for k = 0:N
    for l = 0:N
      # Compute the derivative of the non-linear term
      if (k + l <= N) && (l > 0)
        Df_x[k + 1, l + 1] = - 2.0 * lambda * (x[abs(k - l) + 1] + x[k + l + 1])
      else
        Df_x[k + 1, l + 1] = - 2.0 * lambda * x[abs(k - l) + 1]
      end
    end

    # Add derivative of linear term to diagonal
    Df_x[k + 1, k + 1] += lambda - k^2
  end

  return Df_x
end

Df (generic function with 1 method)

Next we implement Newton's method to find the approximate zeros of $f$.

In [4]:
using LinearAlgebra # Load the LinearAlgebra package

function Newton(f, Df, x0, lambda, Max_Itr, Err_Tol)
  x1 = x0 # Initialize x1 as the initial point x0

  f_x1 = f(x1, lambda)   # Compute f at the initial point
  norm_f_x1 = norm(f_x1) # Norm of f at the initial point

  num_itr = 0       # Intialize iteration counter

  # Return x1 if norm of f_x1 is less than tolerance
  if norm_f_x1 < Err_Tol
    return x1
  end

  # Next start Newton iterations
  while num_itr < Max_Itr
    x0 = x1                 # Set x0 as x1
    f_x0 = f_x1             # Set f_x0 as f_x1

    Df_x0 = Df(x0, lambda)  # Jacobian matrix at x0
    u = Df_x0 \ f_x0        # Newton iteration (solve linear sistem Df_x0 * u = f_x0)
    x1 = x0 - u             # Newton iteration (compute x1)

    f_x1 = f(x1, lambda)    # Compute f at x1
    norm_f_x1 = norm(f_x1)  # Norm of f at x1

    num_itr = num_itr + 1   # Increment iteration counter

    # Check for convergence
    if norm_f_x1 < Err_Tol
      return x1
    end
  end

  # Newton did not converged if we got here
  error("Max iterations exceeded.")
end

Newton (generic function with 1 method)

Now we define the parameter $\lambda$, the dimension $N$, and an initial guess $x_0$ for Newton's method.

In [20]:
# Set the parameter values
lambda = 50; N = 100;

# Initial guess for Newton
x0 = zeros(N + 1, 1); x0[1] = 1.0; x0[2] = 0.3;

In [21]:
x0

101×1 Array{Float64,2}:
 1.0
 0.3
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮  
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

Next we apply Newton's method to find a numerical solution.

In [22]:
max_itr = 100;    # Max number of Newton iterations
err_tol = 1e-10; # Error tolerance for Newton

# Compute x_bar using Newton's method
x_bar = Newton(f, Df, x0, lambda, max_itr, err_tol);

In [23]:
x_bar

101×1 Array{Float64,2}:
  0.4592118385393281    
 -1.3526044416863016e-13
 -1.5889085680809754e-13
 -1.8951668840114127e-13
  0.33369804377891443   
  4.826895994875968e-13 
  1.9822601565776015e-13
  4.6477926992998824e-14
 -0.1095637150530719    
 -3.3300265093877266e-13
 -8.823441783880749e-14 
  3.007467729954672e-14 
  0.027727208655880107  
  ⋮                     
 -5.460429238111271e-26 
  8.080702871186014e-27 
  3.0636948687612145e-26
  7.539210439172542e-17 
  1.0683984359179834e-26
 -1.53342449101171e-27  
 -5.972927894745216e-27 
 -1.3282640593996197e-17
 -2.0875023347567144e-27
  2.837758681937103e-28 
  1.1565202815326492e-27
  2.3348066056079902e-18

Finally we define $A$ and the bounds needed for the radii polynomial

In [24]:
A = inv(Df(x_bar, lambda));

In [25]:
Y0 = norm(A * f(x_bar, lambda), 1)

2.841972686990746e-12

In [26]:
Z0 = norm(I - A * Df(x_bar, lambda), 1)

3.1383775001942832e-12

In [27]:
Z2 = 2.0 * abs(lambda) * norm(A, 1)

8652.19193125087

The radii polynomial is given by $p(r) = Z_2 r^2 - (1 - Z_0) r + Y_0$.

In [28]:
p = Poly([Y0, -(1 - Z0), Z2])

Poly(2.841972686990746e-12 - 0.9999999999968616*x + 8652.19193125087*x^2)

Finally we find the roots of the radii polynomial.

In [29]:
roots(p)

2-element Array{Float64,1}:
 0.00011557764591370954
 2.841972756881768e-12 